In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pyphi
import networkx as nx
import os
import seaborn as sns
import pandas as pd
import time
import random
import pickle
import time
import itertools
from scipy.stats import pearsonr



Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [2]:
df = pd.DataFrame()

#3
#nodes = ('A', 'B', 'C')
#4
#nodes = ('A', 'B', 'C', 'D')
#5
nodes = ['A', 'B', 'C', 'D', 'E']
#6
#nodes = ['A', 'B', 'C', 'D', 'E', 'F']
#7
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
#8
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
#9
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
#10
#nodes = ['A', 'B', 'C', 'D', 'E', 'F','G', 'H', 'I', 'J']
#11
#nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K']
n = len(nodes)

In [3]:
drop_list = []
cm = []
n = len(nodes)
x = n**2-n
y = 2**(x)
seed = []
seed = np.random.randint(1,y+1,dtype = 'int64')

In [4]:
def make_cm(nodes):
    n = len(nodes)
    x = n**2-n
    y = 2**(x)
    seed = np.random.randint(1,y+1,dtype = 'int64')
    binary = bin(seed)
    
    ##seedを2進数に変換し、りすと作成
    d = []
    for i in range(2,len(binary)):
        d.append(int(binary[i]))

    while len(d) != x:
        d.insert(0,0)

    cm_list = []
    cm = []
    
    zeros = np.zeros((n,n), dtype = int)
    s = 0
    for j in range(n):
        for k in range(n):
            if j != k:
                zeros[j][k] = d[s]
                s +=1

    x = []
    axis0 = zeros.sum(axis=0)
    axis1 = zeros.sum(axis=1)
    for l in range(n):
        if axis0[l] == 0 and axis1[l] == 0:       
            x.append(l)
    
            
    if len(x) == 0:
        drop_list = 0
        cm = zeros
    else:
        drop_list = zeros
        cm = 0

        
    return drop_list, cm

In [5]:
drop_list, cm = make_cm(nodes)

In [6]:
while len(df) < 80:
    drop_list, cm = make_cm(nodes)
    if type(cm) != int:
        d = cm.reshape(1,n**2)
        v = d[0]
        v = v.tolist()
        vc = pd.Series(v)
        df = df.append(vc, ignore_index=True)
    df = df.drop_duplicates()

In [7]:
cm_list = []
for i in range(80):
    cm_list.append(df.values[i].reshape(n,n).astype(int))

In [8]:
edge = []
edges = {}
for i in range(80):    
        a = cm_list[i]
        for col in range(len(nodes)):
            for row in range(len(nodes)):
                if a[col][row] == 1:
                    edge.append((nodes[col],nodes[row]))
        edges[i] = edge
        edge = []

# cm,edgeを保存

In [9]:
cm_path = "./cm/{0}_nodes_cm.npy".format(len(nodes))
edge_path = "./edge/{0}_nodes_edge.json".format(len(nodes))

In [10]:
np.save(cm_path, cm_list)
with open(edge_path, 'wb') as fp:
    pickle.dump(edges, fp)

# cm,edgeの読み込み

In [11]:
cm_list = np.load(cm_path)
with open(edge_path, 'rb') as fp:
    edges = pickle.load(fp)

# stateごとのΦ保存

In [12]:
states = np.zeros((pow(2, len(nodes)),len(nodes)), dtype=int)
for i in range(pow(2, len(nodes))):
    for j in range(1, len(bin(i))-1):
        states[i][j-1] = bin(i)[j * (-1)]

In [14]:
success_dic = {}
success_list = []
phis_path = "./phi/{0}_nodes_phis.json".format(len(nodes))
success_state_path = "./success/{0}_nodes_success_state.json".format(len(nodes))

In [16]:
phis = {}
phi = []
for i in range(80):
    tpm = np.dot(states, cm_list[i])  
    for j in range(0,tpm.shape[0]):
        for l in range(0,tpm.shape[1]):
            if tpm[j][l] >= 1:
                tpm[j][l] = 1
    print(i,"cm")
    
    
    
    network = pyphi.Network(tpm, cm=cm_list[i], node_labels=nodes)
    for j in range(pow(2, len(nodes))):
        init_state = states[j]
        try:
            print(j, "states")
            subsystem = pyphi.Subsystem(network, init_state)
            phi.append(pyphi.compute.phi(subsystem))
            sia = pyphi.compute.sia(subsystem)
            sia_path = "./sia/{0}_node/{1}/{2}_sia.pickle".format(len(nodes),i,j)
            with open(sia_path, mode='wb') as f:
                pickle.dump(sia, f)
            #セイリツシタトキノstate
            success_list.append(j)
            
        except:
            
            pass
    success_dic[i] = success_list
    success_list = []
    phis[i] = phi
    phi = []
    
with open(phis_path, 'wb') as fp:
    pickle.dump(phis, fp)
    
with open(success_state_path, 'wb') as fp:
    pickle.dump(success_dic, fp)

0 cm
0 states


1 states
2 states


3 states
4 states


5 states


6 states


...

23 states


24 states
25 states


26 states
27 states


28 states
29 states


30 states
31 states


In [94]:
phis

{10: [0.46875,
  0.012777,
  0.025829,
  0.012777,
  0.012777,
  0.012777,
  0.012777,
  0.012777,
  0.012777,
  0.012777],
 11: [0.25,
  0.055485,
  0.057717,
  0.036033,
  0.057717,
  0.057717,
  0.036033,
  0.036033],
 12: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 13: [0.125,
  0.005102,
  0.067517,
  0.005102,
  0.005102,
  0.005102,
  0.005102,
  0.067517,
  0.005102,
  0.005102],
 14: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 15: [0.21875,
  0.208227,
  0.029337,
  0.029337,
  0.03986,
  0.029337,
  0.21875,
  0.208227,
  0.029337,
  0.029337,
  0.03986,
  0.029337],
 16: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 17: [0.0, 0.0, 0.0, 0.0, 0.0],
 18: [0.3125,
  0.3125,
  0.196181,
  0.196181,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163,
  0.033163],
 19: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 20: [0.179688,
  0.098767,
  0.01841,
  0.01841,
  0.089653,
  0.01841,
  0.057664,
  0.01841],
 21: [0.0, 0.0, 

In [32]:
success_dic

{0: [0, 6], 1: [0, 1, 2, 3, 5, 7], 2: [0, 3, 4, 7]}

In [46]:
with open('test.pickle', mode='wb') as f:
    pickle.dump(sia, f)

In [47]:
with open('test.pickle', mode='rb') as f:
    test2 = pickle.load(f)

In [73]:
with open('./sia/3_node/1/0_sia.pickle', mode='rb') as f:
    p = pickle.load(f)

In [75]:
phis

{0: [0.0, 0.0], 1: [0.4375, 0.215278, 0.4375, 0.215278, 0.215278, 0.215278]}

In [91]:
for i in range(10,30):
    print(i)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
